In [1]:
# Importer les bibliothèques nécessaires
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
# Configuration des chemins
data_dir = '../data/train'

In [3]:
# Générer les données
data = ImageDataGenerator(validation_split=0.20)

train_ds = data.flow_from_directory(data_dir,
                                    class_mode="categorical",
                                    target_size=(229, 229),
                                    batch_size=32,
                                    shuffle=True,
                                    subset='training',
                                    seed=42)

validation = data.flow_from_directory(data_dir,
                                      class_mode="categorical",
                                      target_size=(229, 229),
                                      batch_size=32,
                                      shuffle=True,
                                      subset='validation',
                                      seed=42)


Found 4116 images belonging to 3 classes.
Found 1028 images belonging to 3 classes.


In [4]:
# Construire le modèle
IMG_SIZE = (229, 229)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model.trainable = False

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
# Afficher le résumé du modèle
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ ?                      │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,049,571 (15.45 MB)

In [6]:
# Définir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, min_delta=0.001, verbose=1)


In [8]:
# Entraîner le modèle
history = model.fit(train_ds, epochs=15, validation_data=validation, callbacks=[early_stopping])


c:\Users\heses\OneDrive\Documents\Deep_Learning\env\deep_learning\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


129/129 ━━━━━━━━━━━━━━━━━━━━ 583s 4s/step - accuracy: 0.8375 - loss: 0.3903 - val_accuracy: 0.9484 - val_loss: 0.1647
Restoring model weights from the end of the best epoch: 1.


In [9]:
# Sauvegarder le modèle entraîné
model.save('../model/model_final.h5')